In [1]:
#Import the required libraries

import math
import numpy as np
import pandas as pd
import cv2
import time

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

import cv2
import os
from os import listdir
from tqdm import tqdm
import time
import csv
import pickle

In [2]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
#Check OS path
import shutil
if os.path.isdir('/mnt/fast0/pp837/test_file_haze/.ipynb_checkpoints'):
    shutil.rmtree('/mnt/fast0/pp837/test_file_haze/.ipynb_checkpoints')
else:
    print("Error")

Error


In [4]:
os.path.exists('/mnt/fast0/pp837/test_file_haze/test_38310.jpg')

False

In [5]:
class Node(object):
    def __init__(self,x,y,value):
        self.x = x
        self.y = y
        self.value = value

    def printInfo(self):
        print('%s:%s:%s' %(self.x,self.y,self.value))
        
def getMinChannel(img):
    """
    Check for 3 RGB channels. 3 channels are not present then print an error for the image
    """  

    if len(img.shape)==3 and img.shape[2]==3:
        pass
    else:
        print("bad image shape, input must be color image")
        return None
    
    return np.min(img, axis=2)

def getDarkChannel(img,blockSize = 3):
    """
    Get the dark channel prior in the (RGB) image data.
    """

    if len(img.shape)==2:
        pass
    else:
        print("bad image shape, input image must be two demensions")
        return None

    # blockSize
    if blockSize % 2 == 0 or blockSize < 3:
        print('blockSize is not odd or too small')
        return None

    # addSize
    A = int((blockSize-1)/2) #AddSize

    #New height and new width
    H = img.shape[0] + blockSize - 1
    W = img.shape[1] + blockSize - 1

    imgMiddle = 255 * np.ones((H,W))    

    imgMiddle[A:H-A, A:W-A] = img
    
    imgDark = np.zeros_like(img, np.uint8)    
    
    localMin = 255
    for i in range(A, H-A):
        for j in range(A, W-A):
            x = range(i-A, i+A+1)
            y = range(j-A, j+A+1)
            imgDark[i-A,j-A] = np.min(imgMiddle[x,y])                            
            
    return imgDark

def getAtomsphericLight(darkChannel,img,meanMode = False, percent = 0.001):
    """
    Get the atmosphere light in the (RGB) image data
    """

    size = darkChannel.shape[0]*darkChannel.shape[1]
    height = darkChannel.shape[0]
    width = darkChannel.shape[1]

    nodes = []

    for i in range(0,height):
        for j in range(0,width):
            oneNode = Node(i,j,darkChannel[i,j])
            nodes.append(oneNode)	

    nodes = sorted(nodes, key = lambda node: node.value,reverse = True)

    atomsphericLight = 0

    if int(percent*size) == 0:
        for i in range(0,3):
            if img[nodes[0].x,nodes[0].y,i] > atomsphericLight:
                atomsphericLight = img[nodes[0].x,nodes[0].y,i]
        return atomsphericLight

    if meanMode:
        sum = 0
        for i in range(0,int(percent*size)):
            for j in range(0,3):
                sum = sum + img[nodes[i].x,nodes[i].y,j]
        atomsphericLight = int(sum/(int(percent*size)*3))
        return atomsphericLight

    for i in range(0,int(percent*size)):
        for j in range(0,3):
            if img[nodes[i].x,nodes[i].y,j] > atomsphericLight:
                atomsphericLight = img[nodes[i].x,nodes[i].y,j]
    return atomsphericLight


def getRecoverScene(img, omega=0.95, t0=0.1, blockSize=15, meanMode=False, percent=0.001, refine=True):

    imgGray = getMinChannel(img)
    imgDark = getDarkChannel(imgGray, blockSize = blockSize)
    atomsphericLight = getAtomsphericLight(imgDark,img,meanMode = meanMode,percent= percent)

    imgDark = np.float64(imgDark)
    transmission = 1 - omega * imgDark / atomsphericLight

    transmission[transmission<0.1] = 0.1     
    
    if refine:        
        normI = (img - img.min()) / (img.max() - img.min())  # normalize I
        transmission = guided_filter(normI, transmission, r=40, eps=1e-3)

    sceneRadiance = np.zeros(img.shape)
    img = np.float64(img)
    
    for i in range(3):        
        SR = (img[:,:,i] - atomsphericLight)/transmission + atomsphericLight
               
        SR[SR>255] = 255
        SR[SR<0] = 0                    
        sceneRadiance[:,:,i] = SR  
            
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance

In [6]:
"""Implementation for Guided Image Filtering
Reference:
http://research.microsoft.com/en-us/um/people/kahe/eccv10/
"""

from itertools import combinations_with_replacement
from collections import defaultdict

import numpy as np
from numpy.linalg import inv

R, G, B = 0, 1, 2  # index for convenience

def boxfilter(I, r):
    """Fast box filter implementation.
    Parameters
    ----------
    I:  a single channel/gray image data normalized to [0.0, 1.0]
    r:  window radius
    Return
    -----------
    The filtered image data.
    """        
    M, N = I.shape
    dest = np.zeros((M, N))

    # cumulative sum over Y axis
    sumY = np.cumsum(I, axis=0)
    # difference over Y axis
    dest[:r + 1] = sumY[r: 2 * r + 1]
    dest[r + 1:M - r] = sumY[2 * r + 1:] - sumY[:M - 2 * r - 1]
    dest[-r:] = np.tile(sumY[-1], (r, 1)) - sumY[M - 2 * r - 1:M - r - 1]

    # cumulative sum over X axis
    sumX = np.cumsum(dest, axis=1)
    # difference over Y axis
    dest[:, :r + 1] = sumX[:, r:2 * r + 1]
    dest[:, r + 1:N - r] = sumX[:, 2 * r + 1:] - sumX[:, :N - 2 * r - 1]
    dest[:, -r:] = np.tile(sumX[:, -1][:, None], (1, r)) - \
        sumX[:, N - 2 * r - 1:N - r - 1]

    return dest

def guided_filter(I, p, r=40, eps=1e-3):
    """Refine a filter under the guidance of another (RGB) image.
    Parameters
    -----------
    I:   an M * N * 3 RGB image for guidance.
    p:   the M * N filter to be guided
    r:   the radius of the guidance
    eps: epsilon for the guided filter
    Return
    -----------
    The guided filter
    """    
    M, N = p.shape
    base = boxfilter(np.ones((M, N)), r)

    # each channel of I filtered with the mean filter
    means = [boxfilter(I[:, :, i], r) / base for i in range(3)]
    # p filtered with the mean filter
    mean_p = boxfilter(p, r) / base
    # filter I with p then filter it with the mean filter
    means_IP = [boxfilter(I[:, :, i] * p, r) / base for i in range(3)]
    # covariance of (I, p) in each local patch
    covIP = [means_IP[i] - means[i] * mean_p for i in range(3)]

    # variance of I in each local patch: the matrix Sigma in ECCV10 eq.14
    var = defaultdict(dict)
    for i, j in combinations_with_replacement(range(3), 2):
        var[i][j] = boxfilter(
            I[:, :, i] * I[:, :, j], r) / base - means[i] * means[j]

    a = np.zeros((M, N, 3))
    for y, x in np.ndindex(M, N):
        #         rr, rg, rb
        # Sigma = rg, gg, gb
        #         rb, gb, bb
        Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
        cov = np.array([c[y, x] for c in covIP])
        a[y, x] = np.dot(cov, inv(Sigma + eps * np.eye(3)))  # eq 14

    # ECCV10 eq.15
    b = mean_p - a[:, :, R] * means[R] - \
        a[:, :, G] * means[G] - a[:, :, B] * means[B]

    # ECCV10 eq.16
    q = (boxfilter(a[:, :, R], r) * I[:, :, R] + boxfilter(a[:, :, G], r) *
         I[:, :, G] + boxfilter(a[:, :, B], r) * I[:, :, B] + boxfilter(b, r)) / base

    return q

In [27]:
import re

df_train = pd.read_csv('/mnt/fast0/pp837/train_label.csv')
def findDigit(fname):
    x = re.findall("[0-9]",fname)
    return  int(''.join(x)) 

df_train['image_num'] = df_train['image_name'].apply(lambda x: findDigit(x))
df_train = df_train.set_index('image_num')
df_train.head(5)

,image_name,tags
image_num,,
3577,train_3577.jpg,haze primary
10327,train_10327.jpg,clear primary
1243,train_1243.jpg,clear primary water
17066,train_17066.jpg,clear primary
15959,train_15959.jpg,clear primary


In [28]:
Hazy_img_idx = list(df_train.index)
print(Hazy_img_idx)

[3577, 10327, 1243, 17066, 15959, 32931, 4151, 37844, 28283, 35805, 8340, 31493, 33209, 3796, 33479, 35763, 1520, 27688, 24424, 18721, 13746, 2356, 5375, 34554, 39394, 23075, 7695, 15709, 35084, 4546, 35196, 13615, 22760, 32694, 4106, 32499, 29441, 6246, 9123, 31148, 40413, 17685, 27734, 9998, 38614, 11738, 11363, 364, 35701, 31049, 5701, 4809, 20769, 33166, 8409, 31551, 24864, 6623, 9283, 35137, 30164, 18525, 27759, 1214, 5005, 24985, 30357, 9940, 316, 9708, 9357, 36168, 27203, 37944, 36626, 28912, 11961, 2666, 19671, 4077, 20190, 746, 33644, 1098, 10430, 31711, 11240, 3648, 4391, 13490, 24226, 31555, 18023, 38786, 4839, 3191, 1623, 5792, 23681, 25874, 16007, 4265, 14225, 35800, 4611, 28219, 19933, 31934, 17131, 29159, 21106, 22816, 8666, 7091, 1879, 1501, 11728, 26357, 31250, 9876, 7976, 21046, 7878, 15957, 17601, 28673, 30900, 13724, 21709, 14536, 37069, 15447, 34432, 4112, 34316, 26972, 11765, 26985, 32396, 22224, 27448, 21401, 29752, 2104, 15724, 18341, 1544, 12170, 12427, 4527, 2

In [ ]:
#Hazy_img_idx = [104, 3007, 3794, 23710, 38469]

destination_path = '/mnt/fast0/pp837/train_file_haze/'

for i in tqdm(Hazy_img_idx):    
     
    path = '/mnt/fast0/pp837/train_file/'
    filename = 'train_{}.jpg'.format(i)
    #print(filename)
    img = cv2.imread(path+filename)  #0-255    
    
    dehazed_img2 = getRecoverScene(img, refine=False)
    
    cv2.imwrite(destination_path+filename, dehazed_img2) 

 16%|█▌        | 5039/32383 [2:27:01<14:30:10,  1.91s/it]

In [31]:
#dehazed_img2

In [7]:
import re

df_test = pd.read_csv('/mnt/fast0/pp837/test_label.csv')
def findDigit(fname):
    x = re.findall("[0-9]",fname)
    return  int(''.join(x)) 

df_test['image_num'] = df_test['image_name'].apply(lambda x: findDigit(x))
df_test = df_test.set_index('image_num')
df_test.head(5)

,image_name,tags
image_num,,
32752,test_32752.jpg,clear primary
7767,test_7767.jpg,clear primary
4254,test_4254.jpg,agriculture clear cultivation primary
23119,test_23119.jpg,clear primary
22101,test_22101.jpg,agriculture clear primary


In [8]:
Hazy_img_idx = list(df_test.index)
print(Hazy_img_idx)

[32752, 7767, 4254, 23119, 22101, 26818, 25502, 35301, 29796, 36349, 39181, 38672, 1353, 1239, 28043, 13875, 29040, 26094, 28267, 29544, 36552, 26969, 17707, 35714, 30466, 28342, 684, 10897, 30198, 27234, 16828, 26542, 8075, 38271, 29741, 22872, 24927, 6540, 835, 39508, 31520, 19573, 26531, 24610, 31247, 14405, 3031, 7827, 39213, 5398, 5204, 3525, 5506, 34310, 23136, 29173, 1409, 1521, 31214, 2874, 5395, 37112, 24265, 30112, 31764, 5106, 8295, 23594, 38807, 9121, 32871, 3882, 21912, 6342, 24686, 10622, 4865, 31432, 30696, 31941, 24895, 17311, 9079, 10168, 14347, 18360, 38091, 4398, 38331, 14710, 21991, 31117, 30835, 18385, 32259, 207, 1568, 2382, 20285, 4888, 33352, 23468, 6650, 12425, 10320, 27140, 21728, 2229, 34356, 1288, 7067, 1475, 25602, 35193, 8832, 32729, 7739, 15053, 21288, 5503, 11147, 21561, 21926, 20556, 19309, 34434, 18972, 9296, 37147, 31544, 36117, 559, 32763, 5921, 3931, 810, 22198, 24682, 14317, 36555, 21248, 12565, 4800, 838, 30442, 16324, 33874, 25871, 7425, 32944, 8

In [9]:
#Hazy_img_idx = [104, 3007, 3794, 23710, 38469]

destination_path = '/mnt/fast0/pp837/test_file_haze/'

for i in tqdm(Hazy_img_idx):    
     
    path = '/mnt/fast0/pp837/test_file/'
    filename = 'test_{}.jpg'.format(i)
    #print(filename)
    img = cv2.imread(path+filename)  #0-255    
    
    dehazed_img2 = getRecoverScene(img, refine=False)
    
    cv2.imwrite(destination_path+filename, dehazed_img2) 

100%|██████████| 8096/8096 [3:50:19<00:00,  1.71s/it]  
